In [ ]:
!pip install pyspark

In [ ]:
import pyspark 

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
import pandas as pd
news_popularity = pd.read_csv(io.BytesIO(uploaded['OnlineNewsPredition_Reduced.csv']))

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import time
import timeit



In [ ]:
spark = SparkSession.builder.appName('DDA_Assignment').getOrCreate()

In [ ]:
spark

In [ ]:
df_pyspark = spark.read.csv('OnlineNewsPredition_Reduced.csv')

In [ ]:
df_pyspark = spark.read.option('header', 'true').csv('OnlineNewsPredition_Reduced.csv', inferSchema= True)

In [ ]:
type(df_pyspark)

In [ ]:
df_pyspark.printSchema()

In [ ]:
feature_columns = [column for column in df_pyspark.columns if column != 'shares']

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")


# Splitting train test data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Convert the PySpark DataFrame to a Pandas DataFrame
df_pandas = df_pyspark.toPandas()

# Separate the features (X) and the target variable (y)
target_column = 'shares'
X = df_pandas.drop(target_column, axis=1)
y = df_pandas[target_column]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Random Forest

In [ ]:
# WE try to use the cross validation without pyspark but it requires huge computational resourses. So we are not able to run it

In [ ]:
feature_columns = df_pyspark.columns[:-1]  # assuming 'shares' is the last column in your DataFrame
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")


In [ ]:
rf = RandomForestRegressor(labelCol="shares", featuresCol="features", seed=42, numTrees=500) # Using 500 trees to train our model



In [ ]:
train_data, test_data = df_pyspark.randomSplit([0.8, 0.2], seed=42)


In [ ]:
start_time = time.time()
pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(train_data)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time}")

In [ ]:
predictions = model.transform(test_data)


In [ ]:
start_time = time.time()
evaluator = RegressionEvaluator(labelCol="shares", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time}")

In [ ]:
evaluator_mae = RegressionEvaluator(labelCol="shares", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error (MAE):", mae)
